# Adi2Azi

Author: Jacob Humble - AI5ZK

This notebook will turn an .adi file into an azimuthal map centered on the configured origin_lat and origin_lon. The .adi file should be put in the same directory as this file. Note that the file **must** include, for each QSO, a LAT and LON field. wsjt-x's ADIF does not contain this, and will not work correctly if you upload it. Any .adi produced by QRZ *will* include this data, so one *possible* way to get a compatible file is to upload your log to QRZ, then export it.

In [1]:
# Coords to center map on
origin_lat = 0
origin_lon = 0

In [2]:
# Read .adi file and turn it into lists of coordinates

import pandas as pd
import adif_io as ad
with open('log.adi', mode='rt', encoding="utf8") as file:
    data = file.read()

qsos_raw, header = ad.read_from_string(data)
qsos = pd.DataFrame(qsos_raw)

lolat, lolon = qsos.apply(lambda row : ad.degrees_from_location(row['LAT']), axis=1), qsos.apply(lambda row : ad.degrees_from_location(row['LON']), axis=1)

# MatPlotLib - BaseMap Version

This outputs a QSO map using MatPlotLib and Basemap.

In [ ]:
# MatPlotLib Basemap Version

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
width = 28305550
m = Basemap(width=width,height=width,projection='aeqd',
            lat_0=origin_lat,lon_0=origin_lon)

# fill background.
m.drawmapboundary(fill_color='ghostwhite')

# draw coasts
m.drawcoastlines(linewidth=0.5)

# 20 degree graticule.
m.drawparallels(np.arange(-80,81,20))
m.drawmeridians(np.arange(-180,180,20))

# plot qsos
lonpnt, latpnt = m(lolon, lolat)
m.plot(lonpnt,latpnt,'rx', markersize=1) 

plt.title('Azimuthal Map of QSOs')
plt.show()

# GeoPandas

This outputs an azimuthal QSO map using GeoPandas.

In [ ]:
import geopandas as gpd
import geodatasets
from shapely.geometry import Point

# Setup World Map
world = gpd.read_file(geodatasets.get_path("naturalearth.land"))
fig, ax = plt.subplots(figsize = (15,15))


# Setup QSO Points
qso_geo = [Point(xy) for xy in zip(lolon, lolat)]
qso_df = gpd.GeoDataFrame(geometry = qso_geo)
qso_df.crs = 4326

# Set QTH Point
origin = [Point(origin_lon, origin_lat)]
o_df = gpd.GeoDataFrame(geometry = origin)
o_df.crs = 4326

# Transform & Plot
trnsfrm_rule = '+proj=laea +lat_0=' + str(origin_lat) + ' +lon_0=' + str(origin_lon) + ' +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs'

world = world.to_crs(trnsfrm_rule)
world.plot(ax = ax)

o_df = o_df.to_crs(trnsfrm_rule)
origin = o_df.plot(ax = ax, markersize = 30, color = 'yellow', marker = 'o',label = 'You')

qso_df = qso_df.to_crs(trnsfrm_rule)
q = qso_df.plot(ax = ax, markersize = 20, color = 'red',marker = 'x',label = 'QSOs')


plt.title('Azimuthal Map of QSOs')
plt.legend()
plt.show()
